#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [131]:
# Dhruv Arora - da32895

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [132]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [133]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [134]:
def get_data(filename):
    data = []
    
    # read the file and parse the data
    with open(filename, 'r') as f:
      for line in f:
        # remove the newline character
        if (line[-1] == '\n'):
           line = line[:-1]

        # split the line into a list of floats
        tokenized = line.split(',')
        tokenized = list(map(float, tokenized))

        # create DataPoint
        data.append(DataPoint(tokenized[-1], tokenized[:-1]))

    return data

In [135]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [136]:
def prediction_helper(root, dp):
    # base case
    if root.is_leaf:
        return root.prediction
    
    # check left or right, and go
    f_val = dp.features[root.feature_idx]

    if f_val < root.thresh_val:
        return prediction_helper(root.left_child, dp)
    else:
        return prediction_helper(root.right_child, dp)

def make_prediction(tree_root, data_point):
    # use recursive helper
    return prediction_helper(tree_root, data_point)

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [137]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []

    # for each check against threshold
    for d in data:
        if d.features[feature_idx] < threshold:
            left_split.append(d)
        else:
            right_split.append(d)

    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [138]:
# returns counts of 0s and 1s in data's labels
def get_value_counts(data):
    counts = dict()
    counts[0] = 0
    counts[1] = 0

    for d in data:
        counts[d.label] += 1

    return counts

def calc_entropy(data):
    entropy = 0.0
    counts = get_value_counts(data)
    total = counts[0] + counts[1]
    a = counts[0] / total # Pi, i=0
    b = counts[1] / total # Pi, i=1

    if a == 0 or b == 0:
        return 0
    
    # following formula
    entropy += -a * log2(a)
    entropy += -b * log2(b)

    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [139]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    
    # sort by the feature
    d = sorted(data, key=lambda row: row.features[feature_idx])
    entropy_total = calc_entropy(data)

    # go through all possible thresholds
    for i in range(len(d) - 1):
        # skip unnecessary thresholds
        prev, next = d[i].features[feature_idx], d[i+1].features[feature_idx]
        if prev == next:
            continue

        thresh = (prev + next) / 2
        left_set, right_set = split_dataset(data, feature_idx, thresh)

        left_entropy = calc_entropy(left_set)
        left_weight = len(left_set) / len(data)

        right_entropy = calc_entropy(right_set)
        right_weight = len(right_set) / len(data)
        
        entropy_split = (left_weight * left_entropy) + (right_weight * right_entropy)

        # compare split to before split and best split so far
        gain = entropy_total - entropy_split
        if gain > best_info_gain:
            best_info_gain = gain
            best_thresh = thresh

    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [140]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
    best_info_gain = -1

    # find which feature has the best gain @ the best threshold
    for f_idx in range(len(data[0].features)):
        gain, thresh = calc_best_threshold(data, f_idx)
        if gain > best_info_gain:
            best_feature = f_idx
            best_thresh = thresh
            best_info_gain = gain

    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [141]:
def create_leaf_node(data):
    counts = get_value_counts(data)

    node = TreeNode()
    node.is_leaf = True
    # prediction is the majority class
    node.prediction = 0 if counts[0] > counts[1] else 1

    return node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [142]:
def create_dt_helper(data, levels):
    # base case
    if levels <= 1:
        return create_leaf_node(data)

    # if pure, return leaf
    if calc_entropy(data) == 0:
        return create_leaf_node(data)

    # greedy --> try best split every time
    f_idx, thresh = identify_best_split(data)

    # create internal node
    root = TreeNode()
    root.is_leaf = False
    root.feature_idx = f_idx
    root.thresh_val = thresh
    
    left_set, right_set = split_dataset(data, f_idx, thresh)

    # recursively create left and right children
    root.left_child = create_dt_helper(left_set, levels - 1)
    root.right_child = create_dt_helper(right_set, levels - 1)

    return root

def create_decision_tree(data, max_levels):
    # just in case
    if max_levels < 1:
        return None

    return create_dt_helper(data, max_levels)

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [143]:
# calc percentage of correct test results
def accuracy_helper(root, test_set):
    correct = 0.0
    
    for test in test_set:
        prediction = make_prediction(root, test)
        if prediction == test.label:
            correct += 1
    
    return correct / len(test_set)

# 5-fold cross validation
def calc_accuracy(tree_root, data):
    fold_len = int(len(data) / 5)
    total_accuracy = 0.0

    # each fold
    for fold in range(5):
        print('STARTING FOLD %d' % fold)

        # define test and training sets
        test_set = data[fold_len * fold: fold_len * (fold + 1)]
        training_set = [x for x in data if x not in test_set]

        # create tree and calc accuracy
        root = create_decision_tree(training_set, 10)
        accuracy = accuracy_helper(root, test_set)
        total_accuracy += accuracy
        print('Fold %d had an accuracy of %f' % (fold, accuracy))


    return total_accuracy / 5.0

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [144]:
import time

d = get_data("messidor_features.txt")

# the timer is just for fun! you will NOT be graded on runtime
start = time.time()

# create the decision tree
tree = create_decision_tree(d, 10)

end = time.time()
print ('Time taken:', end - start)

# calculate the accuracy of the tree
accuracy = calc_accuracy(tree, d)
print ('The total accuracy of the tree is ', str(accuracy * 100.0))
# tree.printTree()

Time taken: 8.191265106201172
STARTING FOLD 0
Fold 0 had an accuracy of 0.639130
STARTING FOLD 1
Fold 1 had an accuracy of 0.630435
STARTING FOLD 2
Fold 2 had an accuracy of 0.669565
STARTING FOLD 3
Fold 3 had an accuracy of 0.639130
STARTING FOLD 4
Fold 4 had an accuracy of 0.643478
The total accuracy of the tree is  64.43478260869566
